<a href="https://colab.research.google.com/github/chaeyeonlee1/lemontree725.io/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving validation.csv to validation.csv


In [15]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00


In [7]:
import numpy as np
import sys
import os
import torch
import datasets
import torch.nn.functional as F
import tqdm
import pandas as pd
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

from sklearn.metrics import accuracy_score

In [8]:
if torch.cuda.is_available():
  device = "cuda"
  print("Running on GPU!")
else:
  device = "cpu"
  print("Running on CPU D:")

Running on GPU!


In [9]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [10]:
model = model.to(device)

In [11]:
def load_dataset(file):
  df = pd.read_csv(file, sep='\t')
  df = df.dropna(subset=['text', 'label'])

  char_len = []
  for i in tqdm.tqdm(df["text"]):
    char_len.append(len(i.split()))

  print(f"# of Training Examples: {len(df)}")
  print(f"Average text size: {np.mean(char_len)} words")
  print(f"Maximum text size: {max(char_len)} words, Minimum: {min(char_len)} words")

  dataset = Dataset.from_pandas(df)
  dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
  dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

  return dataset

In [14]:
train_data = load_dataset("train.csv")

100%|██████████| 10680/10680 [00:00<00:00, 548731.74it/s]

# of Training Examples: 10680
Average text size: 19.67443820224719 words
Maximum text size: 129 words, Minimum: 1 words


  0%|          | 0/11 [00:00<?, ?ba/s]

In [16]:
valid_data = load_dataset("validation.csv")

100%|██████████| 1187/1187 [00:00<00:00, 356380.73it/s]

# of Training Examples: 1187
Average text size: 19.525695029486098 words
Maximum text size: 67 words, Minimum: 1 words


  0%|          | 0/2 [00:00<?, ?ba/s]

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=train_data,         # training dataset
        eval_dataset=valid_data,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()   

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, Unnamed: 0. If __index_level_0__, text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10680
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1670
  Number of trainable parameters = 109484547


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.011222,0.994945
2,0.064400,0.006773,0.996630
3,0.004700,0.011919,0.997473
4,0.004700,0.016801,0.996630
5,0.000800,0.012181,0.997473


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, Unnamed: 0. If __index_level_0__, text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1187
  Batch size = 32
Saving model checkpoint to temp/checkpoint-334
Configuration saved in temp/checkpoint-334/config.json
Model weights saved in temp/checkpoint-334/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, Unnamed: 0. If __index_level_0__, text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1187
  Batch size = 32
Saving model checkpoint to temp/checkpoint-668
Configur

TrainOutput(global_step=1670, training_loss=0.02092775793824189, metrics={'train_runtime': 1154.2826, 'train_samples_per_second': 46.262, 'train_steps_per_second': 1.447, 'total_flos': 3512564126668800.0, 'train_loss': 0.02092775793824189, 'epoch': 5.0})

In [23]:
test_data = load_dataset("test.csv")

100%|██████████| 2967/2967 [00:00<00:00, 368704.08it/s]

# of Training Examples: 2967
Average text size: 19.739467475564542 words
Maximum text size: 89 words, Minimum: 1 words


  0%|          | 0/3 [00:00<?, ?ba/s]

In [24]:
model.eval()
trainer.predict(test_data).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, Unnamed: 0. If __index_level_0__, text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2967
  Batch size = 32


{'test_loss': 0.016453567892313004,
 'test_accuracy': 0.9966295921806538,
 'test_runtime': 20.6406,
 'test_samples_per_second': 143.746,
 'test_steps_per_second': 4.506}

In [25]:
trainer.save_model('finbert-opinionfact/')

Saving model checkpoint to finbert-opinionfact/
Configuration saved in finbert-opinionfact/config.json
Model weights saved in finbert-opinionfact/pytorch_model.bin
